In [17]:
import pandas as pd

In [100]:
bom = pd.read_csv('bom_oscars.csv')

bafta = pd.read_csv('bafta.csv')
bafta['Year'] = bafta['Year'] - 1 # Year is offset by 1. Need to correct

dga = pd.read_csv('dga.csv')[['Year', 'Movie']] # Don't need the director column

pga = pd.read_csv('pga.csv')

gg_drama = pd.read_csv('gg_drama.csv')
gg_drama['Year'] = gg_drama['Year'] - 1

gg_comedy = pd.read_csv('gg_comedy.csv')
gg_comedy['Year'] = gg_comedy['Year'] - 1

In [76]:
# Merge with BAFTA awards
df1 = bom.merge(bafta, how='left', 
                left_on=bom['Movie'].apply(lambda x: x.lower()),
                right_on=bafta['Movie'].apply( lambda x: x.lower()),
                suffixes=['', '_bafta'])

df1['BAFTA'] = df1['Movie_bafta'].isnull().apply(lambda x: 0 if x else 1)
del df1['Movie_bafta'], df1['Year_bafta']

In [79]:
# Merge with DGA awards
df2 = df1.merge(dga, how='left',
               left_on=df1['Movie'],
               right_on=dga['Movie'],
               suffixes=['', '_dga'])

df2['DGA'] = df2['Movie_dga'].isnull().apply(lambda x: 0 if x else 1)
del df2['Movie_dga'], df2['Year_dga']

In [91]:
# Merge with PGA awards
df3 = df2.merge(pga, how='left',
               left_on=df2['Movie'],
               right_on=pga['Movie'],
               suffixes=['', '_pga'])

df3['PGA'] = df3['Movie_pga'].isnull().apply(lambda x: 0 if x else 1)
del df3['Movie_pga'], df3['Year_pga']

In [104]:
# Merge with GG Comedy awards
df4 = df3.merge(gg_comedy, how='left',
               left_on=df3['Movie'],
               right_on=gg_comedy['Movie'],
               suffixes=['', '_ggc'])

df4['GGC'] = df4['Movie_ggc'].isnull().apply(lambda x: 0 if x else 1)
del df4['Movie_ggc'], df4['Year_ggc']

In [106]:
# Merge with GG Drama awards
df5 = df4.merge(gg_drama, how='left',
               left_on=df4['Movie'],
               right_on=gg_drama['Movie'],
               suffixes=['', '_ggd'])

df5['GGD'] = df5['Movie_ggd'].isnull().apply(lambda x: 0 if x else 1)
del df5['Movie_ggd'], df5['Year_ggd']

In [109]:

oscars = df5

In [110]:
def normalize_column_by_year(df, column):
    normalized = df.groupby(['Year']).apply(lambda x: x[column] / x[column].mean())
    normalized.index = [x[1] for x in normalized.index]
    
    return normalized

oscars['Post-Nom Gross'] = normalize_column_by_year(oscars, 'Post-Nom Gross')
oscars['Pre-Nom Gross'] = normalize_column_by_year(oscars, 'Pre-Nom Gross')
oscars['Post-Nom Theatres'] = normalize_column_by_year(oscars, 'Post-Nom Theatres')
oscars['Pre-Nom Theatres'] = normalize_column_by_year(oscars, 'Pre-Nom Theatres')

In [112]:
oscars.to_csv('final_dataset.csv', index=False)

In [101]:
# Test to see if PGA and DGA awards are oscar nominated movies

for year in df1['Year'].unique():
    
    ggd_winner = gg_drama[gg_drama['Year'] == year]['Movie']
    ggc_winner = gg_comedy[gg_comedy['Year'] == year]['Movie']
    oscar_noms = df1[df1['Year'] == year]['Movie']
    
    print('------- {:} -------'.format(year))
    if len(ggd_winner) > 0:
        print("GGDrama Winner: ", ggd_winner.iloc[0], ". Nominated for oscars:", ggd_winner.iloc[0] in oscar_noms.tolist())
        
    if len(ggc_winner) > 0:
        print("GGComedy Winner: ", ggc_winner.iloc[0], ". Nominated for oscars:", ggc_winner.iloc[0] in oscar_noms.tolist())
        
    print(oscar_noms)
    print()

------- 1982 -------
GGDrama Winner:  E. T.: The Extra-Terrestrial . Nominated for oscars: False
GGComedy Winner:  Tootsie . Nominated for oscars: True
0    E.T.: The Extra-Terrestrial
1                        Tootsie
2                    The Verdict
3                         Gandhi
4                        Missing
Name: Movie, dtype: object

------- 1983 -------
GGDrama Winner:  Terms of Endearment . Nominated for oscars: True
GGComedy Winner:  Yentl . Nominated for oscars: False
5    Terms of Endearment
6          The Big Chill
7        The Right Stuff
8         Tender Mercies
9            The Dresser
Name: Movie, dtype: object

------- 1984 -------
GGDrama Winner:  Amadeus . Nominated for oscars: True
GGComedy Winner:  Romancing The Stone . Nominated for oscars: False
10                Amadeus
11    Places in the Heart
12     The Killing Fields
13     A Passage to India
14      A Soldier's Story
Name: Movie, dtype: object

------- 1985 -------
GGDrama Winner:  Out Of Africa . Nomina

In [45]:
dga_winner

'Gandhi'

True

'Gandhi'